## Computational Human Reading Prediction

The study of eye movement is of great interest to neuroscience, since they reflect cognitive processes that underlie visual tasks, in particularly reading. An important variable for determining these movements is called Predictibility. That probably needs a better name. This, Predictability, reprents the prediction that one is making about the coming word during reading. In fields such as Neurolinguistics, this Predictibility variable is not estimated from a part of the text but from responses from other reading filling in the word that follows given a the same context. 

In parallel, the field of NLP has estimated this type of prediction in an automatic manner as some of their goals. A simple but at the same time successful example includes n-grams. Where the probability to predict a word is constructed from the appearance of the context in a large corpus of text, that represents the knowledge of the language that the reader has.  

This model is able to extend the utility of recently read text for realizing such probabilities (cache n-gram). In this work it is proposed to estimate the predictability of a word in an automatic form using parts from distinct variants of the language models. For our dataset we show that the new automatic predictibility is equally as effective as the predictibility of human explained eye movements, is much better to understand, cheaper and more rapid to obtain as it does not require experiments that involve a great count of people. 

In [1]:
import numpy as np
import pandas as pd
from os import walk 
from tqdm import tqdm

In [2]:
filepath="../data/LMM-CBP/csv_in"
data=[]

# Va a caminar en las carpetas para cargar los files
for root, dirs, files in tqdm(walk(filepath)):
    for file in files:
        temp=pd.read_csv(f"{filepath}/{file}", delimiter=";")
        data.append(temp)

del temp

df=pd.concat(data)
print(df.shape)


0it [00:00, ?it/s]C:\Users\JDOli\AppData\Local\Temp\ipykernel_11284\3333485044.py:5: DtypeWarning: Columns (95) have mixed types. Specify dtype option on import or set low_memory=False.
  temp=pd.read_csv(f"{filepath}/{file}", delimiter=";")
1it [00:22, 22.61s/it]


(2586948, 140)


In [3]:
df.describe()

,suj_id,n_orac,palnum,tipo,pred,freq,length,MaxJump,bad_epoch,stopword,...,E119,E120,E121,E122,E123,E124,E125,E126,E127,E128
count,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,...,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586948e+06,2.586947e+06
mean,1.450000e+01,9.485284e+01,4.375697e+00,1.016722e+00,-2.119133e-01,4.377367e+04,4.341137e+00,3.199554e-01,6.005733e-01,4.760312e-01,...,-9.378470e-02,-3.066079e-03,-4.883013e-02,1.872346e-02,-4.121687e-02,-4.585046e-02,4.162723e-02,1.097454e-01,1.410374e-01,1.750807e-01
std,8.077749e+00,4.925550e+01,2.373298e+00,8.517471e-01,9.609049e-01,6.940815e+04,2.360397e+00,1.217291e+00,4.897807e-01,4.994253e-01,...,1.199427e+01,1.287864e+01,1.201152e+01,1.230401e+01,1.280992e+01,1.357826e+01,1.460180e+01,1.268019e+01,1.212173e+01,1.277105e+01
min,1.000000e+00,1.100000e+01,1.000000e+00,0.000000e+00,-1.255300e+00,0.000000e+00,1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,...,-5.808102e+02,-7.236652e+02,-7.258200e+02,-7.262158e+02,-7.530132e+02,-7.490185e+02,-7.527536e+02,-7.541962e+02,-7.518157e+02,-7.520677e+02
25%,7.750000e+00,5.400000e+01,2.000000e+00,0.000000e+00,-1.255300e+00,1.940000e+02,2.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,...,-6.574130e+00,-6.429716e+00,-6.499453e+00,-6.574852e+00,-6.698232e+00,-6.586955e+00,-7.274955e+00,-6.762504e+00,-6.425458e+00,-6.377016e+00
50%,1.450000e+01,9.300000e+01,4.000000e+00,1.000000e+00,-3.010300e-01,2.718000e+03,4.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,-1.456375e-01,-1.038615e-01,-8.447389e-02,-2.715446e-02,-3.462234e-02,-1.620925e-02,-3.477534e-03,3.852869e-02,9.089095e-02,8.002501e-02
75%,2.125000e+01,1.380000e+02,6.000000e+00,2.000000e+00,6.989700e-01,6.221400e+04,6.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,...,6.298971e+00,6.258519e+00,6.357286e+00,6.577438e+00,6.600107e+00,6.479055e+00,7.341053e+00,6.925016e+00,6.657640e+00,6.611491e+00
max,2.800000e+01,1.970000e+02,1.200000e+01,2.000000e+00,1.255300e+00,2.647210e+05,1.200000e+01,2.000000e+00,1.000000e+00,1.000000e+00,...,1.967297e+02,1.282204e+03,1.885576e+02,2.219662e+02,1.895408e+02,3.634467e+02,2.147726e+02,1.707255e+02,1.692756e+02,2.543207e+02


In [4]:
# Va a reconstruir los frases base en la apracion punta
frase=""
sents=[]
for word in df.loc[df.suj_id==1][' pal'].to_list():
    if frase=="":
        frase=word
    elif word.find(".")>-1:
        frase=frase+" "+word
        sents.append(frase)
        frase=""
    else:
        frase=frase+" "+word

df_phrases=pd.DataFrame({"sentences":sents})
df_phrases.head()

,sentences
0,La picadura de ciertas arañas puede ser mortal.
1,Cuando hay hambre no hay pan duro.
2,La película terminó de forma extraña.
3,El gato atrapó muchos ratones.
4,Sobre gustos no hay nada escrito.


In [9]:
# Guarda a csv
# df_phrases.to_csv("sentences.csv", index=0)

In [66]:
# Va a traer un parte del df
df_subset=df[
    [
        'suj_id',
        ' pal',
        ' palnum',
        ' freq',
        ' length',
        ' time',
    ]
].copy()
df_subset.columns=["suj_id", "pal", "palnum", "freq", "length", "time"]

In [70]:
# Va a identificar frases base en una punta aparicion
# No va a identificar el mismo frase con el mismo identificacion de subjetos diferentes.
sent_grp_count=0
sent_id=[]
for i,row in tqdm(df_subset.iterrows()):
     sent_id.append(sent_grp_count)
     word=row.pal
     word_period=word.find(".")
     if word_period>-1:
          sent_grp_count+=1

In [67]:
df_subset.groupby(
    ["suj_id", ""]
)

,suj_id,pal,palnum,freq,length,time
0,1,La,1,192476,2,-101.5625
1,1,picadura,2,9,8,-101.5625
2,1,de,3,264721,2,-101.5625
3,1,ciertas,4,380,7,-101.5625
4,1,arañas,5,20,6,-101.5625
...,...,...,...,...,...,...
25111,28,la,5,192476,2,664.0625
25112,28,panadería,6,10,9,664.0625
25113,28,cocinan,7,7,7,664.0625
25114,28,el,8,139594,2,664.0625
